In [3]:
import pandas as pd
import numpy as np
import os

np.random.seed(42)

n = 1000

os.makedirs("data", exist_ok=True)

data = {
    "device_type": np.random.choice(["Ventilator", "Monitor", "Infusion Pump"], n),
    "usage_hours": np.random.randint(100, 20000, n),
    "device_age": np.random.randint(1, 10, n),
    "temperature_avg": np.random.uniform(30, 90, n),
    "error_count": np.random.randint(0, 50, n),
    "last_maintenance_days": np.random.randint(1, 365, n),
    "vibration_level": np.random.uniform(0.1, 5.0, n),
}

df = pd.DataFrame(data)

df["maintenance_needed"] = (
    (df["usage_hours"] > 12000) |
    (df["error_count"] > 20) |
    (df["temperature_avg"] > 70) |
    (df["last_maintenance_days"] > 180)
).astype(int)

df.to_csv("data/maintenance_data.csv", index=False)

print("Dataset generated successfully!")


Dataset generated successfully!


In [4]:
from google.colab import files

files.download("data/maintenance_data.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score

# Load data
df = pd.read_csv("data/maintenance_data.csv")

X = df.drop("maintenance_needed", axis=1)
y = df["maintenance_needed"]

# Columns
categorical_features = ["device_type"]
numerical_features = [
    "usage_hours",
    "device_age",
    "temperature_avg",
    "error_count",
    "last_maintenance_days",
    "vibration_level"
]

# Preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numerical_features)
    ]
)

# Model
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=10,
    random_state=42
)

# Pipeline
pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("model", model)
])

# Train / test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Train
pipeline.fit(X_train, y_train)

# Evaluate
y_pred = pipeline.predict(X_test)
y_proba = pipeline.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("ROC AUC:", roc_auc_score(y_test, y_proba))

# Save model
import os
os.makedirs("model", exist_ok=True)

# Save model
joblib.dump(pipeline, "model/model.pkl")
print("Model saved successfully!")



Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.94      0.97        18
           1       0.99      1.00      1.00       182

    accuracy                           0.99       200
   macro avg       1.00      0.97      0.98       200
weighted avg       1.00      0.99      0.99       200

ROC AUC: 0.9996947496947497
Model saved successfully!


In [6]:
from fastapi import FastAPI
from pydantic import BaseModel
import joblib
import os
import numpy as np
import pandas as pd

# Load model
model_path = "model/model.pkl"
model = joblib.load(model_path)

app = FastAPI(
    title="Medical Equipment Predictive Maintenance API",
    description="Predict if a medical device needs maintenance soon",
    version="1.0"
)

# Input schema
class DeviceData(BaseModel):
    device_type: str
    usage_hours: int
    device_age: int
    temperature_avg: float
    error_count: int
    last_maintenance_days: int
    vibration_level: float

# Prediction endpoint
@app.post("/predict")
def predict(data: DeviceData):
    # Convert input to DataFrame
    df = pd.DataFrame([data.dict()])

    # Make prediction
    prediction = model.predict(df)[0]
    probability = model.predict_proba(df)[0][1]

    return {
        "maintenance_needed": int(prediction),
        "risk_probability": float(round(probability, 2))
    }

# Health check
@app.get("/health")
def health():
    return {"status": "API is running!"}


In [7]:
# Install dependencies
!pip install fastapi[all] -q

from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.testclient import TestClient
import joblib
import pandas as pd

# Load trained model
model = joblib.load("model/model.pkl")

# Create FastAPI app
app = FastAPI(title="Medical Equipment Predictive Maintenance API")

# Input schema
class DeviceData(BaseModel):
    device_type: str
    usage_hours: int
    device_age: int
    temperature_avg: float
    error_count: int
    last_maintenance_days: int
    vibration_level: float

# Prediction endpoint
@app.post("/predict")
def predict(data: DeviceData):
    df = pd.DataFrame([data.model_dump()])
    prediction = model.predict(df)[0]
    probability = model.predict_proba(df)[0][1]
    return {"maintenance_needed": int(prediction), "risk_probability": float(round(probability, 2))}

# Health check
@app.get("/health")
def health():
    return {"status": "API is running!"}

# Test the API using TestClient (inside Colab)
client = TestClient(app)

# Example request
data_example = {
    "device_type": "Ventilator",
    "usage_hours": 15000,
    "device_age": 5,
    "temperature_avg": 75.2,
    "error_count": 12,
    "last_maintenance_days": 200,
    "vibration_level": 2.1
}

response = client.post("/predict", json=data_example)
print("Predict response:", response.json())

health_resp = client.get("/health")
print("Health response:", health_resp.json())






   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.4/57.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.1/331.1 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.7/517.7 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.8/456.8 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.6/821.6 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 959.8/959.8 kB 45.0 MB/s eta 0:00:00
Predict response: {'maintenance_needed': 1, 'risk_probability': 1.0}
Health response: {'status': 'API is running!'}


In [8]:
!ls


data  model  sample_data


In [9]:
!pip install plotly -q

import pandas as pd
import plotly.express as px

# Load dataset
df = pd.read_csv("data/maintenance_data.csv")

# Example: distribution of risk probability (simulated)
df["maintenance_needed"] = (
    (df["usage_hours"] > 12000) |
    (df["error_count"] > 20) |
    (df["temperature_avg"] > 70) |
    (df["last_maintenance_days"] > 180)
).astype(int)

# Plot
fig = px.histogram(df, x="maintenance_needed", title="Maintenance Needed Distribution")
fig.show()

# Scatter example: usage_hours vs error_count
fig2 = px.scatter(df, x="usage_hours", y="error_count", color="maintenance_needed",
                  title="Usage Hours vs Error Count")
fig2.show()


In [11]:
import logging

# إعداد logging
logging.basicConfig(filename="model_logs.log",
                    level=logging.INFO,
                    format="%(asctime)s - %(levelname)s - %(message)s")

@app.post("/predict")
def predict(data: DeviceData):
    df = pd.DataFrame([data.model_dump()])
    prediction = model.predict(df)[0]
    probability = model.predict_proba(df)[0][1]

    # سجل كل prediction
    logging.info(f"Input: {data.model_dump()} | Prediction: {int(prediction)}, Probability: {probability:.2f}")

    return {"maintenance_needed": int(prediction), "risk_probability": float(round(probability, 2))}


In [12]:
!pip install plotly ipywidgets -q
import pandas as pd
import plotly.express as px
from ipywidgets import interact

# Load dataset
df = pd.read_csv("data/maintenance_data.csv")

# Interactive histogram: اختر العمود اللي تحب تعرضه
@interact(column=df.columns.tolist())
def interactive_hist(column):
    fig = px.histogram(df, x=column, color="maintenance_needed",
                       title=f"Distribution of {column} by Maintenance Needed",
                       marginal="box",  # show boxplot
                       nbins=30)
    fig.show()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.2 MB/s eta 0:00:00


interactive(children=(Dropdown(description='column', options=('device_type', 'usage_hours', 'device_age', 'tem…

In [13]:
!pip install streamlit -q


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 34.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.0 MB/s eta 0:00:00


In [16]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px

st.title("Medical Equipment Predictive Maintenance Dashboard")

# Load data
df = pd.read_csv("data/maintenance_data.csv")

# Sidebar filters
device_type = st.sidebar.multiselect("Device Type", df['device_type'].unique(), default=df['device_type'].unique())
df_filtered = df[df['device_type'].isin(device_type)]

# Histogram
fig = px.histogram(df_filtered, x="usage_hours", color="maintenance_needed", nbins=30,
                   title="Usage Hours Distribution by Maintenance Needed")
st.plotly_chart(fig)

# Scatter plot
fig2 = px.scatter(df_filtered, x="usage_hours", y="error_count", color="maintenance_needed",
                  title="Usage Hours vs Error Count")
st.plotly_chart(fig2)


Writing app.py


In [17]:
!streamlit run app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.87.233:8501

  Stopping...


In [18]:
!pip uninstall -y streamlit


Found existing installation: streamlit 1.52.2
Uninstalling streamlit-1.52.2:
  Successfully uninstalled streamlit-1.52.2
